<a href="https://colab.research.google.com/github/Raulespz/emotion_ai/blob/main/Emotion_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the necessary packages

import pandas as pd
import numpy as np
import os
import PIL
import seaborn as sns
import pickle
from PIL import *
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.python.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow

In [ ]:
keyfacial_df = pd.read_csv('/content/data.csv')
keyfacial_df.head()

FileNotFoundError: ignored

In [ ]:
keyfacial_df.isnull().sum()

In [ ]:
keyfacial_df.shape

In [ ]:
keyfacial_df['Image'].shape

In [ ]:
keyfacial_df['Image'] = keyfacial_df['Image'].apply(lambda x: np.fromstring(x, dtype = int, sep = ' ').reshape(96, 96))

In [ ]:
keyfacial_df['Image'][0].shape

In [ ]:
keyfacial_df.describe()

In [ ]:
# keyfacial_df['Image'] = keyfacial_df['Image'].apply(lambda x: np.fromstring(x, dtype = int, sep = ' ').reshape(96, 96))

In [ ]:
#This one is in replace in the code of the upper cell due do not run:
import numpy as np
import pandas as pd

# Assuming 'keyfacial_df' is your DataFrame with the 'Image' column

# Define a function to handle the reshaping and handle the ValueError
def reshape_image(x):
    try:
        return np.fromstring(x, dtype=int, sep=' ').reshape(96, 96)
    except ValueError:
        # Handle the error here (e.g., return a default value, raise an exception, etc.)
        return np.zeros((96, 96))  # Default value as zeros

# Apply the reshaping function to the 'Image' column
keyfacial_df['Image'] = keyfacial_df['Image'].apply(reshape_image)

In [ ]:
keyfacial_df['Image'][0].shape

In [ ]:
i = np.random.randint(1, len(keyfacial_df)) # i code generates a random integer i between 1 and the length of the DataFrame keyfacial_df.
plt.imshow(keyfacial_df['Image'][i], cmap = 'gray')
for j in range(1, 31, 2):
       plt.plot(keyfacial_df.loc[i][j-1], keyfacial_df.loc[i][j], 'rx')

#for j in range(1, 31, 2): is a loop that iterates over a range of numbers.
#In this case, it starts from 1, ends at 31 (exclusive), and increments by 2
#in each iteration. So, j will take values 1, 3, 5, ..., 29.

#keyfacial_df.loc[i][j-1] accesses the value at the index i
 #(which was randomly generated earlier) and column j-1 of the DataFrame
# keyfacial_df. This code snippet selects a specific element from the
# DataFrame based on the generated index i and the current value of j.

#keyfacial_df.loc[i][j] accesses the value at the index i and column
#j of the DataFrame keyfacial_df. Similar to the previous step, this
#code snippet selects another specific element from the DataFrame.

#plt.plot(keyfacial_df.loc[i][j-1], keyfacial_df.loc[i][j], 'rx')
#creates a scatter plot using matplotlib's plot function. It plots
#a single point with coordinates (keyfacial_df.loc[i][j-1],
#                                 keyfacial_df.loc[i][j]) and uses a red 'x'
#                                 marker ('rx') to represent the point.

In [ ]:
# Let's view more images in a grid format
fig = plt.figure(figsize=(20, 20))

for i in range(64):
  ax = fig.add_subplot(8, 8, i + 1)
  image = plt.imshow(keyfacial_df['Image'][i], cmap = 'gray')
  for j in range(1, 31, 2):
       plt.plot(keyfacial_df.loc[i][j-1], keyfacial_df.loc[i][j], 'rx')

In [ ]:
import random
# Let's view more images in a grid format
fig = plt.figure(figsize=(20, 20))

for i in range(64):
    k = random.randint(1, len(keyfacial_df))
    ax = fig.add_subplot(8, 8, i + 1)
    image = plt.imshow(keyfacial_df['Image'][k],cmap = 'gray')
    for j in range(1,31,2):
        plt.plot(keyfacial_df.loc[k][j-1], keyfacial_df.loc[k][j], 'rx')

In [ ]:
#Create a new copy of the dataframe

import copy
keyfacial_df_copy = copy.copy(keyfacial_df)

In [ ]:
#Obtain the columns in the dataframe
columns = keyfacial_df_copy.columns[:-1] # this takes of the columns of our dataframe but not the last one
columns

In [ ]:
#Horizontal Flip - flip the images along y axis

#In simple terms, this line of code horizontally flips each image in the 'Image'
#column of the DataFrame keyfacial_df_copy. It applies the np.flip() function from
#NumPy to reverse the order of the elements along the horizontal axis (columns)
#of each image, effectively flipping the image horizontally.
#The modified images are then stored back in the 'Image' column of the DataFrame.

keyfacial_df_copy['Image'] = keyfacial_df_copy['Image'].apply(lambda x: np.flip(x, axis = 1))

# since we are flipping horizontally, y coordinate values would be the same
# Only x coordiante values would change, all we have to do is to subtract our initial x-coordinate values from width of the image(96)
for i in range(len(columns)):
  if i%2 == 0:
    keyfacial_df_copy[columns[i]] = keyfacial_df_copy[columns[i]].apply(lambda x: 96. - float(x) )

In [ ]:
#Since we are flipping horizontally, y coordinate values would be the same
#Only x coordinate values would change, all we have to do is to substract our initial x-coordinate values from width of the
###for i in range(len(columns)):
###  if i%2 == 0:
###    keyfacial_df_copy[columns[i]] = keyfacial_df_copy[columns[i]].apply(lambda x: 96. - float(x))

In [ ]:
# Show the original image
plt.imshow(keyfacial_df['Image'][9], cmap = 'gray')
for j in range(1, 31, 2):
       plt.plot(keyfacial_df.loc[9][j-1], keyfacial_df.loc[9][j], 'rx')

In [ ]:
# Show the Horizontally flipped image
plt.imshow(keyfacial_df_copy['Image'][9], cmap = 'gray')
for j in range(1, 31, 2):
       plt.plot(keyfacial_df.loc[9][j-1], keyfacial_df.loc[9][j], 'rx')

In [ ]:
# Concatenate the original dataframe with the augmented dataframe it means that we are going to duplicate our dataframe
# with teh original dataframe and the copy that we already created to this augmented instance.
augmented_df = np.concatenate((keyfacial_df, keyfacial_df_copy))
augmented_df.shape

In [ ]:
# Randomingly increasing the brightness of the images
# We multiply pixel vlaues by random valkues between 1.5 and 2 to increase the brightness of the image
# We clip the value between 0 to 255

import random

keyfacial_df_copy = copy.copy(keyfacial_df)
keyfacial_df_copy['Image'] = keyfacial_df_copy['Image'].apply(lambda x:np.clip(random.uniform(1.5, 2)* x, 0.0, 255.0))
augmented_df = np.concatenate((augmented_df, keyfacial_df_copy))
augmented_df.shape

In [ ]:
# Show Image with increased brightness
plt.imshow(keyfacial_df_copy['Image'][9], cmap = 'gray')
for j in range(1, 31, 2):
       plt.plot(keyfacial_df_copy.loc[9][j-1], keyfacial_df_copy.loc[9][j], 'rx')

In [ ]:
keyfacial_df_copy = copy.copy(keyfacial_df)

In [ ]:
keyfacial_df_copy['Image'] = keyfacial_df_copy['Image'].apply(lambda x: np.flip(x, axis = 0))

for i in range(len(columns)):
  if i%2 == 1:
    keyfacial_df_copy[columns[i]] = keyfacial_df_copy[columns[i]].apply(lambda x: 96. - float(x) )

In [ ]:
plt.imshow(keyfacial_df_copy['Image'][0], cmap='gray')
for j in range(1, 31, 2):
        plt.plot(keyfacial_df_copy.loc[0][j-1], keyfacial_df_copy.loc[0][j], 'rx')

In [ ]:
# Obtain the value of images which is present in the 31st column (since index start from 0, we refer to 31st column by 30)
img = augmented_df[:,30]

# Normalize the images
img = img/255.

# Create an empty array of shape (x, 96, 96, 1) to feed the model
X = np.empty((len(img), 96, 96, 1))

# Iterate through the img list and add image values to the empty array after expanding it's dimension from (96, 96) to (96, 96, 1)
for i in range(len(img)):
  X[i,] = np.expand_dims(img[i], axis = 2)

# Convert the array type to float32
X = np.asarray(X).astype(np.float32)
X.shape

In [ ]:
# Obtain the value of x & y coordinates which are to used as target.
y = augmented_df[:,:30]
y = np.asarray(y).astype(np.float32)
y.shape

In [ ]:
# Split the data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
def res_block(X, filter, stage):

  # Convolutional_block
  X_copy = X

  f1 , f2, f3 = filter

  # Main Path
  X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_conv_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = MaxPool2D((2,2))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_a')(X)
  X = Activation('relu')(X)

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_conv_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_b')(X)
  X = Activation('relu')(X)

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_c')(X)


  # Short path
  X_copy = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_copy', kernel_initializer= glorot_uniform(seed = 0))(X_copy)
  X_copy = MaxPool2D((2,2))(X_copy)
  X_copy = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_copy')(X_copy)

  # ADD
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

  # Identity Block 1
  X_copy = X


  # Main Path
  X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_1_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_a')(X)
  X = Activation('relu')(X)

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_1_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_b')(X)
  X = Activation('relu')(X)

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_1_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_c')(X)

  # ADD
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

  # Identity Block 2
  X_copy = X


  # Main Path
  X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_2_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_a')(X)
  X = Activation('relu')(X)

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_2_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_b')(X)
  X = Activation('relu')(X)

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_2_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_c')(X)

  # ADD
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

  return X

In [ ]:
input_shape = (96, 96, 1)

# Input tensor shape
X_input = Input(input_shape)

# Zero-padding
X = ZeroPadding2D((3,3))(X_input)

# 1 - stage
X = Conv2D(64, (7,7), strides= (2,2), name = 'conv1', kernel_initializer= glorot_uniform(seed = 0))(X)
X = BatchNormalization(axis =3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3,3), strides= (2,2))(X)

# 2 - stage
X = res_block(X, filter= [64,64,256], stage= 2)

# 3 - stage
X = res_block(X, filter= [128,128,512], stage= 3)


# Average Pooling
X = AveragePooling2D((2,2), name = 'Averagea_Pooling')(X)

# Final layer
X = Flatten()(X)
X = Dense(4096, activation = 'relu')(X)
X = Dropout(0.2)(X)
X = Dense(2048, activation = 'relu')(X)
X = Dropout(0.1)(X)
X = Dense(30, activation = 'relu')(X)


model_1_facialKeyPoints = Model( inputs= X_input, outputs = X)
model_1_facialKeyPoints.summary()

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model_1_facialKeyPoints.compile(loss = "mean_squared_error", optimizer = 'adam', metrics = ['accuracy'])


In [ ]:
!pip uninstall tensorflow
!pip uninstall keras
!pip install tensorflow
!pip install keras==2.12.0

adam = tf.keras.optimizers.Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)
model_1_facialKeyPoints.compile(loss = "mean_squared_error", optimizer = adam , metrics = ['accuracy'])
# Check this out for more information on Adam optimizer: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam

In [ ]:
# save the best model with least validation loss
checkpointer = ModelCheckpoint(filepath = "FacialKeyPoints_weights.hdf5", verbose = 1, save_best_only = True)

In [ ]:
history = model_1_facialKeyPoints.fit(X_train, y_train, batch_size = 32, epochs = 2, validation_split = 0.05, callbacks=[checkpointer])

In [ ]:
# save the model architecture to json file for future use

model_json = model_1_facialKeyPoints.to_json()
with open("FacialKeyPoints-model.json","w") as json_file:
  json_file.write(model_json)
